---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [15]:
def blight_model():
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import LabelEncoder
    from pandas.api.types import is_string_dtype
    import numpy as np
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.model_selection import train_test_split

    # train_set cleaned of all tickets where the violator was not responsible
    train_df = pd.read_csv("train.csv", encoding = "ISO-8859-1").dropna(how="all", subset=["compliance"], axis=0).set_index("ticket_id", drop=True)
    test_df = pd.read_csv("test.csv", encoding = "ISO-8859-1").set_index("ticket_id", drop=True)

    # columns which include information on compliance (potential data leakage)
    target_info = ["payment_amount", "payment_date", "payment_status", "balance_due", "collection_status", "compliance_detail"]
    train_df.drop(target_info, axis=1, inplace=True)

    def clean_data(df):
        # Data cleaning
        discarded_columns = [
            "grafitti_status", # entirely NaN
            "violation_zip_code", # entirely NaN
            "violator_name", # unusable due to spelling errors and too many alternative naming conventions (for example for big companies)
            "violation_description", # same as above, too many variables in how people describe violations
            "mailing_address_str_number", 
            "mailing_address_str_name", 
            "zip_code", 
            # these columns are discarded, since there is no unified information available for every violator
            # for example: the location of a violator in Detroit is known, but not everywhere else (at least in the available data)
            # in an additional model it might be possible to use this information, for example to check if this is a repeat violator
            # but in the current background (per-ticket evaluation) this info is generally not available
        ]
        df.drop(discarded_columns, axis=1, inplace=True)
        return df

    def preprocess_data(df):
        # filling the missing entries in the test_df here. Usually I would just drop the incomplete row with
        # f.dropna(how="any", axis=0, inplace=True)
        # but autograders are stupid and there is no other way
        df.fillna(method="pad", inplace=True)

        # lowercasing all string entries
        for col in df.columns:
            if is_string_dtype(df[col]):
                df[col] = df[col].str.lower()

        # adding location data
        adresses_df = pd.read_csv("addresses.csv")
        latlons_df = pd.read_csv("latlons.csv")
        latlons_df["reduced_address"] = latlons_df["address"].replace(", Detroit MI(?: [0-9]*)?", "", regex=True).str.lower()

        latlon_lookup = latlons_df[["reduced_address", "lat", "lon"]].set_index("reduced_address")
        latlon_lookup = latlon_lookup[~latlon_lookup.index.duplicated(keep='first')]

        df["combined_violation_location"] = df.apply(lambda x: "{} {}".format(int(x["violation_street_number"]), x["violation_street_name"].lower()), axis=1)
        df = df.merge(latlon_lookup, how="left", left_on="combined_violation_location", right_index=True)
        # df.dropna(subset=["lat", "lon"], how="any", inplace=True)

        df.drop(["violation_street_number", "violation_street_name", "combined_violation_location"], axis=1, inplace=True)

        df["violator_in_detroit"] = (df["city"].str.lower()).str.contains("det(?:roit)?", regex=True)
        df["violator_in_michigan"] = (df["state"] == "mi")
        df["violator_in_usa"] = (df["country"] == "usa")

        df.drop(["city", "state", "country", "non_us_str_code"], axis=1, inplace=True)

        # encoding string labels
        cols_to_encode = ["agency_name", "inspector_name", "violation_code", "disposition"]
        df["violation_code"] = df["violation_code"].str.extract("([0-9\-\.\/]{2,})", expand=False)

        label_encoder = LabelEncoder()

        for col in cols_to_encode:
            df["{}_label".format(col)] = label_encoder.fit_transform(df[col])

        df.drop(cols_to_encode, axis=1, inplace=True)

        # encoding datetime columns
        cols_translate_datetime_to_sine_cosine = ["ticket_issued_date", "hearing_date"]

        for col in cols_translate_datetime_to_sine_cosine:
            df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
            sec_of_day = df[col].dt.hour * 3600 + df[col].dt.minute * 60 + df[col].dt.second
            day_of_year = df[col].dt.dayofyear
            year = df[col].dt.year
            df["{}_sin_sec_of_day".format(col)] = np.sin(2*np.pi*sec_of_day/86400)
            df["{}_cos_sec_of_day".format(col)] = np.cos(2*np.pi*sec_of_day/86400)
            df["{}_sin_day_of_year".format(col)] = np.sin(2*np.pi*day_of_year/365)
            df["{}_cos_day_of_year".format(col)] = np.cos(2*np.pi*day_of_year/365)
            df["{}_year".format(col)] = year

        df.drop(cols_translate_datetime_to_sine_cosine, axis=1, inplace=True)

        # damn this autograder is stupid
        df.fillna(method="pad", inplace=True)
        df = df.apply(pd.to_numeric, errors='coerce', axis=1)

        return df

    train_df = preprocess_data(clean_data(train_df))

    X = train_df.loc[:, train_df.columns != 'compliance']
    y = train_df["compliance"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    gbc = GradientBoostingClassifier(random_state=0, n_estimators=400, min_samples_leaf=1, min_samples_split=0.4, learning_rate=0.1, max_features="auto", max_depth=10).fit(X_train, y_train)

    test_df = preprocess_data(clean_data(test_df))

    result = pd.Series(index=test_df.index, data=gbc.predict_proba(test_df)[:,1], name="compliance", dtype="float32")

    return result

In [ ]:
"""import numpy as np
bm = blight_model()
res = '{:40s}'.format('Object Type:')
res += ['Failed: type(bm) should Series\n','Passed\n'][type(bm)==pd.Series]
res += '{:40s}'.format('Data Shape:')
res += ['Failed: len(bm) should be 61001\n','Passed\n'][len(bm)==61001]
res += '{:40s}'.format('Data Values Type:')
res += ['Failed: bm.dtype should be float\n','Passed\n'][str(bm.dtype).count('float')>0]
res += '{:40s}'.format('Data Values Infinity:')
res += ['Failed: values should not be infinity\n','Passed\n'][not any(np.isinf(bm))]
res += '{:40s}'.format('Data Values NaN:')
res += ['Failed: values should not be NaN\n','Passed\n'][not any(np.isnan(bm))]
res += '{:40s}'.format('Data Values in [0,1] Range:')
res += ['Failed: all values should be in [0.,1.]\n','Passed\n'][all((bm<=1.) & (bm>=0.))]
res += '{:40s}'.format('Data Values not all 0 or 1:')
res += ['Failed: values should be scores not predicted labels\n','Passed\n'][not all((bm.isin({0,1,0.0,1.0})))]
res += '{:40s}'.format('Index Type:')
res += ['Failed: type(bm.index) should be Int64Index\n','Passed\n'][type(bm.index)==pd.Int64Index]
res += '{:40s}'.format('Index Values:')
if bm.index.shape==(61001,):
    res +=['Failed: index values should match test.csv\n','Passed\n'
          ][all(pd.read_csv('readonly/test.csv',usecols=[0],index_col=0
                           ).sort_index().index.values==bm.sort_index().index.values)]
else:
    res+='Failed: bm.index length should be 61001'
res += '{:40s}'.format('Can run model twice:')
bm2 = None
try:
    bm2 = blight_model()
    res += 'Passed\n'
except:
    res += ['Failed: second run of blight_model() threw an Exception']
res += '{:40s}'.format('Can run model twice with same results:')
if not bm2 is None:
    res += ['Failed: second run of blight_model() produced different results (this might not be a problem)\n','Passed\n'][
        all(bm.apply(lambda x:round(x,3))==bm2.apply(lambda x:round(x,3))) and all(bm.index==bm2.index)]    
print(res)"""

In [1]:
"""import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import is_string_dtype
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

# train_set cleaned of all tickets where the violator was not responsible
train_df = pd.read_csv("readonly/train.csv", encoding = "ISO-8859-1").dropna(how="all", subset=["compliance"], axis=0).set_index("ticket_id", drop=True)
test_df = pd.read_csv("readonly/test.csv", encoding = "ISO-8859-1").set_index("ticket_id", drop=True)

# columns which include information on compliance (potential data leakage)
target_info = ["payment_amount", "payment_date", "payment_status", "balance_due", "collection_status", "compliance_detail"]
train_df.drop(target_info, axis=1, inplace=True)

def clean_data(df):
    # Data cleaning
    discarded_columns = [
        "grafitti_status", # entirely NaN
        "violation_zip_code", # entirely NaN
        "violator_name", # unusable due to spelling errors and too many alternative naming conventions (for example for big companies)
        "violation_description", # same as above, too many variables in how people describe violations
        "mailing_address_str_number", 
        "mailing_address_str_name", 
        "zip_code", 
        # these columns are discarded, since there is no unified information available for every violator
        # for example: the location of a violator in Detroit is known, but not everywhere else (at least in the available data)
        # in an additional model it might be possible to use this information, for example to check if this is a repeat violator
        # but in the current background (per-ticket evaluation) this info is generally not available
    ]
    df.drop(discarded_columns, axis=1, inplace=True)
    return df

def preprocess_data(df):
    # filling the missing entries in the test_df here. Usually I would just drop the incomplete row with
    # f.dropna(how="any", axis=0, inplace=True)
    # but autograders are stupid and there is no other way
    df.fillna(method="pad", inplace=True)

    # lowercasing all string entries
    for col in df.columns:
        if is_string_dtype(df[col]):
            df[col] = df[col].str.lower()

    # adding location data
    adresses_df = pd.read_csv("readonly/addresses.csv")
    latlons_df = pd.read_csv("readonly/latlons.csv")
    latlons_df["reduced_address"] = latlons_df["address"].replace(", Detroit MI(?: [0-9]*)?", "", regex=True).str.lower()

    latlon_lookup = latlons_df[["reduced_address", "lat", "lon"]].set_index("reduced_address")
    latlon_lookup = latlon_lookup[~latlon_lookup.index.duplicated(keep='first')]

    df["combined_violation_location"] = df.apply(lambda x: "{} {}".format(int(x["violation_street_number"]), x["violation_street_name"].lower()), axis=1)
    df = df.merge(latlon_lookup, how="left", left_on="combined_violation_location", right_index=True)
    # df.dropna(subset=["lat", "lon"], how="any", inplace=True)

    df.drop(["violation_street_number", "violation_street_name", "combined_violation_location"], axis=1, inplace=True)

    df["violator_in_detroit"] = (df["city"].str.lower()).str.contains("det(?:roit)?", regex=True)
    df["violator_in_michigan"] = (df["state"] == "mi")
    df["violator_in_usa"] = (df["country"] == "usa")

    df.drop(["city", "state", "country", "non_us_str_code"], axis=1, inplace=True)

    # encoding string labels
    cols_to_encode = ["agency_name", "inspector_name", "violation_code", "disposition"]
    df["violation_code"] = df["violation_code"].str.extract("([0-9\-\.\/]{2,})", expand=False)

    label_encoder = LabelEncoder()

    for col in cols_to_encode:
        df["{}_label".format(col)] = label_encoder.fit_transform(df[col])

    df.drop(cols_to_encode, axis=1, inplace=True)

    # encoding datetime columns
    cols_translate_datetime_to_sine_cosine = ["ticket_issued_date", "hearing_date"]

    for col in cols_translate_datetime_to_sine_cosine:
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d %H:%M:%S')
        sec_of_day = df[col].dt.hour * 3600 + df[col].dt.minute * 60 + df[col].dt.second
        day_of_year = df[col].dt.dayofyear
        year = df[col].dt.year
        df["{}_sin_sec_of_day".format(col)] = np.sin(2*np.pi*sec_of_day/86400)
        df["{}_cos_sec_of_day".format(col)] = np.cos(2*np.pi*sec_of_day/86400)
        df["{}_sin_day_of_year".format(col)] = np.sin(2*np.pi*day_of_year/365)
        df["{}_cos_day_of_year".format(col)] = np.cos(2*np.pi*day_of_year/365)
        df["{}_year".format(col)] = year

    df.drop(cols_translate_datetime_to_sine_cosine, axis=1, inplace=True)

    # damn this autograder is stupid
    df.fillna(method="pad", inplace=True)
    df = df.apply(pd.to_numeric, errors='coerce', axis=1)

    return df

train_df = preprocess_data(clean_data(train_df))

X = train_df.loc[:, train_df.columns != 'compliance']
y = train_df["compliance"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
# gbc = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 7, max_features=6, min_samples_leaf=1, min_samples_split= 0.4, n_estimators= 400, random_state=0).fit(X_train, y_train)
gbc = GradientBoostingClassifier(random_state=0, n_estimators=200, learning_rate=0.2, max_features=None, max_depth=5).fit(X_train, y_train)

test_df = preprocess_data(clean_data(test_df))

result = pd.Series(index=test_df.index, data=gbc.predict_proba(test_df)[:,1], name="compliance", dtype="float32")

"""

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
"""from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

test_gbc = GradientBoostingClassifier(random_state=0, n_estimators=200, learning_rate=0.2, max_features=None, max_depth=5).fit(X_train, y_train)
y_scores = test_gbc.decision_function(X_test)

print('Accuracy of GBDT classifier on training set: {:.2f}'
     .format(test_gbc.score(X_train, y_train)))
print('Accuracy of GBDT classifier on test set: {:.2f}'
     .format(test_gbc.score(X_test, y_test)))
print('AUC-ROC of GBDT classifier on test set: {:.2f}\n'
     .format(roc_auc_score(y_test, y_scores)))

test_gbc = GradientBoostingClassifier(random_state=0, n_estimators=200, learning_rate=0.3, max_features=None, max_depth=5).fit(X_train, y_train)
y_scores = test_gbc.decision_function(X_test)

print('Accuracy of GBDT classifier on training set: {:.2f}'
     .format(test_gbc.score(X_train, y_train)))
print('Accuracy of GBDT classifier on test set: {:.2f}'
     .format(test_gbc.score(X_test, y_test)))
print('AUC-ROC of GBDT classifier on test set: {:.2f}\n'
     .format(roc_auc_score(y_test, y_scores)))"""

Accuracy of GBDT classifier on training set: 0.95
Accuracy of GBDT classifier on test set: 0.94
AUC-ROC of GBDT classifier on test set: 0.84



In [ ]:
"""target = "compliance"
target_info = ["payment_amount", "payment_date", "payment_status", "balance_due", "collection_status", "compliance_detail"]
index = "ticket_id"
train_columns = ["agency_name", "inspector_name", "violator_name", "ticket_issued_date", "hearing_date",
                 "violation_code", "violation_description", "disposition",
                 "fine_amount", "admin_fee", "state_fee", "late_fee",
                 "discount_amount", "clean_up_cost", "judgment_amount", "grafitti_status"]

violation_location_columns =["violation_street_number", "violation_street_name", "violation_zip_code"]
violator_location_columns = ["mailing_address_str_number", "mailing_address_str_name", "city", "state", "zip_code", "non_us_str_code", "country"]
discarded_columns = [
    "grafitti_status", # entirely NaN
]

processed_columns = ["agency_name_id", "inspector_name_id", "violator_name"]"""

In [ ]:
"""import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pandas.api.types import is_string_dtype
import numpy as np

# train_set cleaned of all tickets where the violator was not responsible
train_df = pd.read_csv("readonly/train.csv", encoding = "ISO-8859-1").dropna(how="all", subset=["compliance"], axis=0).set_index("ticket_id", drop=True)
test_df = pd.read_csv("readonly/test.csv", encoding = "ISO-8859-1").set_index("ticket_id", drop=True)"""

In [ ]:
"""# Data cleaning
# columns which include information on compliance (potential data leakage)
target_info = ["payment_amount", "payment_date", "payment_status", "balance_due", "collection_status", "compliance_detail"]
train_df.drop(target_info, axis=1, inplace=True)

discarded_columns = [
    "grafitti_status", # entirely NaN
    "violation_zip_code", # entirely NaN
    "violator_name", # unusable due to spelling errors and too many alternative naming conventions (for example for big companies)
    "violation_description", # same as above, too many variables in how people describe violations
    "mailing_address_str_number", 
    "mailing_address_str_name", 
    "zip_code", 
    # these columns are discarded, since there is no unified information available for every violator
    # for example: the location of a violator in Detroit is known, but not everywhere else (at least in the available data)
    # in an additional model it might be possible to use this information, for example to check if this is a repeat violator
    # but in the current background (per-ticket evaluation) this info is generally not available
]
train_df.drop(discarded_columns, axis=1, inplace=True)"""

In [ ]:
"""# PREPROCESSING
# lowercasing all string entries
for col in train_df.columns:
    if is_string_dtype(train_df[col]):
        train_df[col] = train_df[col].str.lower()"""

In [ ]:
"""# adding location data
adresses_df = pd.read_csv("readonly/addresses.csv")
latlons_df = pd.read_csv("readonly/latlons.csv")
latlons_df["reduced_address"] = latlons_df["address"].replace(", Detroit MI(?: [0-9]*)?", "", regex=True).str.lower()

latlon_lookup = latlons_df[["reduced_address", "lat", "lon"]].set_index("reduced_address")
latlon_lookup = latlon_lookup[~latlon_lookup.index.duplicated(keep='first')]

train_df["combined_violation_location"] = train_df.apply(lambda x: "{} {}".format(int(x["violation_street_number"]), x["violation_street_name"].lower()), axis=1)
train_df = train_df.merge(latlon_lookup, how="left", left_on="combined_violation_location", right_index=True)
train_df.dropna(subset=["lat", "lon"], how="any", inplace=True)

train_df.drop(["violation_street_number", "violation_street_name", "combined_violation_location"], axis=1, inplace=True)

train_df["violator_in_detroit"] = (train_df["city"].str.lower()).str.contains("det(?:roit)?", regex=True)
train_df["violator_in_michigan"] = (train_df["state"] == "mi")
train_df["violator_in_usa"] = (train_df["country"] == "usa")

train_df.drop(["city", "state", "country", "non_us_str_code"], axis=1, inplace=True)"""

In [ ]:
"""# encoding string labels
cols_to_encode = ["agency_name", "inspector_name", "violation_code", "disposition"]
train_df["violation_code"] = train_df["violation_code"].str.extract("([0-9\-\.\/]{2,})", expand=False)

# Encode labels
label_encoder = LabelEncoder()

for col in cols_to_encode:
    train_df["{}_label".format(col)] = label_encoder.fit_transform(train_df[col])

train_df.drop(cols_to_encode, axis=1, inplace=True)"""

In [ ]:
"""# encoding datetime columns
cols_translate_datetime_to_sine_cosine = ["ticket_issued_date", "hearing_date"]

for col in cols_translate_datetime_to_sine_cosine:
    train_df.dropna(how="all", subset=[col], axis=0, inplace=True) # drop any rows which dont have this information
    train_df[col] = pd.to_datetime(train_df[col], format='%Y-%m-%d %H:%M:%S')
    sec_of_day = train_df[col].dt.hour * 3600 + train_df[col].dt.minute * 60 + train_df[col].dt.second
    day_of_year = train_df[col].dt.dayofyear
    year = train_df[col].dt.year
    train_df["{}_sin_sec_of_day".format(col)] = np.sin(2*np.pi*sec_of_day/86400)
    train_df["{}_cos_sec_of_day".format(col)] = np.cos(2*np.pi*sec_of_day/86400)
    train_df["{}_sin_day_of_year".format(col)] = np.sin(2*np.pi*day_of_year/365)
    train_df["{}_cos_day_of_year".format(col)] = np.cos(2*np.pi*day_of_year/365)
    train_df["{}_year".format(col)] = year

train_df.drop(cols_translate_datetime_to_sine_cosine, axis=1, inplace=True)"""

In [ ]:
"""from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

X = train_df.loc[:, train_df.columns != 'compliance']
y = train_df["compliance"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
gbc = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 7, max_features=6, min_samples_leaf=1, min_samples_split= 0.4, n_estimators= 400).fit(X_train, y_train)"""

In [ ]:
"""from sklearn.metrics import confusion_matrix

y_pred = gbc.predict(X_test)

confusion_matrix(y_test, y_pred)"""

In [ ]:
"""X_test.columns"""

In [ ]:
"""%matplotlib notebook
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt

y_scores = gbc.decision_function(X_test)

fpr_gbc, tpr_gbc, _ = roc_curve(y_test, y_scores)
roc_auc_gbc = auc(fpr_gbc, tpr_gbc)

plt.plot(fpr_gbc, tpr_gbc)
roc_auc_gbc"""

In [ ]:
"""from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
new_gbc = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 7, max_features=6, min_samples_leaf=1, min_samples_split= 0.4, n_estimators= 400)
cv_scores = cross_val_score(new_gbc, X, y, cv=skf)

print('Cross-validation scores (3-fold):', cv_scores)
print('Mean cross-validation score (3-fold): {:.3f}'
     .format(np.mean(cv_scores)))"""